<a href="https://colab.research.google.com/github/txvuong150764/Synonym-Dictonary/blob/master/Synonym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 63.4 MB/s 
     |████████████████████████████████| 182 kB 100.8 MB/s 


In [2]:
pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 4.2 MB/s 
     |████████████████████████████████| 591 kB 91.8 MB/s 
     |████████████████████████████████| 1.0 MB 88.3 MB/s 
     |████████████████████████████████| 235 kB 67.5 MB/s 


# Mount Drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [4]:
from tensorflow import keras
from keras.layers import Dense
import tensorflow as tf
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from underthesea import ner
import string
import json

In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Install vncorenlp

In [6]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git
!pip install vncorenlp
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP('VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

def word_segment(text):
    word_segmented_text = annotator.tokenize(text) 
    sentences = []
    for sentence in word_segmented_text:
        sentences.append(' '.join(sentence))
    return ' '.join(sentences)

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 233 (delta 12), reused 1 (delta 0), pack-reused 212
Receiving objects: 100% (233/233), 214.22 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.0 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645950 sha256=37c62390432eadf09a239dec91eca85fdb9250a6739a0513d99e4089c62b714d
  Stored in directory: /root/.cache/pip/wheels/3c/e9/86/706911c12e307aeb9a2702399f0dad38d36f1d6f9dde8af35e
Successfully built vncorenlp


# Question Similarity Model

In [7]:
class EquivalenceClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EquivalenceClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [8]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load Question Similarity Model


In [9]:
model = EquivalenceClassifier(n_classes=2)
model.to(device)
model.load_state_dict(torch.load(f'/content/drive/MyDrive/phobert_fold6.pth'))

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

# Load Dictionary and Stopword file


In [ ]:
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Viet74K_Tokenized.txt') as f:
    tmp_dictionary = f.readlines()

with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-stopwords-master/vietnamese-stopwords-dash.txt') as f:
    tmp_stop_word = f.readlines()

In [ ]:
dictionary = []
for word in tmp_dictionary:
    word = word.replace('\n', '')
    dictionary.append(word)

In [ ]:
stop_word = []
for word in tmp_stop_word:
    word = word.replace('\n', '')
    stop_word.append(word)

In [ ]:
punctuation_set = set(string.punctuation)

In [ ]:
trec57_train_df = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/trec57/train.csv')
trec57_test_df = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/trec57/test.csv')
trec57_dev_df = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/trec57/dev.csv')

In [ ]:
trec57_train_df = trec57_train_df.drop(trec57_train_df[trec57_train_df['label'] == 'CONVERSTATION'].index)
trec57_test_df = trec57_test_df.drop(trec57_test_df[trec57_test_df['label'] == 'CONVERSTATION'].index)
trec57_dev_df = trec57_dev_df.drop(trec57_dev_df[trec57_dev_df['label'] == 'CONVERSTATION'].index)

trec57_train_df = trec57_train_df.drop(trec57_train_df[trec57_train_df['label'] == 'BAD'].index)
trec57_test_df = trec57_test_df.drop(trec57_test_df[trec57_test_df['label'] == 'BAD'].index)
trec57_dev_df = trec57_dev_df.drop(trec57_dev_df[trec57_dev_df['label'] == 'BAD'].index)

In [ ]:
trec57_train_df[trec57_train_df['label'] == 'CONVERSTATION']

,label,text


In [ ]:
trec57_train_df = trec57_train_df.drop(['label'], axis=1)
trec57_test_df = trec57_test_df.drop(['label'], axis=1)
trec57_dev_df = trec57_dev_df.drop(['label'], axis=1)

In [ ]:
li = [trec57_train_df, trec57_test_df, trec57_dev_df]

In [ ]:
trec57_question_list = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
trec57_question_list

,text
0,boston ở đâu
1,tiểu bang nào ở phía tây pennsylvania
2,hàn quốc nằm ở khu vực nào
3,quần đảo trường sa được việt nam chia làm mấy cụm
4,thủ đô của indonesia có tên là gì
...,...
21492,canh rau má có tốt cho bà bầu không
21493,ăn bầu có tốt cho bà bầu không
21494,ăn dưa gang có tốt cho bà bầu không
21495,ăn sữa có tốt cho bà bầu không


In [ ]:
wiki_question_list = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/wiki75/wiki74-revising.csv')

In [ ]:
wiki_question_list = wiki_question_list.drop(['g_id', 'id', 'label', 'set'], axis=1)

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/QAC/version/v0.0.1/train.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

qac_question_list = []

for json_str in json_list:
    result = json.loads(json_str)
    if result['question'] not in qac_question_list:
      qac_question_list.append(result['question'])

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/QAC/version/v0.0.1/test.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    if result['question'] not in qac_question_list:
      qac_question_list.append(result['question'])

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/QAC/version/v0.0.1/dev.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    if result['question'] not in qac_question_list:
      qac_question_list.append(result['question'])

In [ ]:
all_question_list = []

# for question in trec57_question_list['text']:
#   all_question_list.append(question)

for question in wiki_question_list['text']:
  all_question_list.append(question)

for question in qac_question_list:
  all_question_list.append(question)

In [ ]:
for question in all_question_list:
  if type(question) != str:
    all_question_list.remove(question)

In [ ]:
all_tokenzied_question_list = []
for i in tqdm(range(len(all_question_list))):
  all_tokenzied_question_list.append(word_segment(all_question_list[i]))

100%|██████████| 30768/30768 [01:10<00:00, 434.82it/s]


In [ ]:
noun_and_question_list = {}
np_and_question_list = {}
nc_and_question_list = {}
p_and_question_list = {}
b_and_question_list = {}
adj_and_question_list = {}
verb_and_question_list = {}

for question in tqdm(all_question_list):
  pos_tag = annotator.pos_tag(question)
  for i in range(len(pos_tag[0])):
    tag = pos_tag[0][i][1]
    if tag == 'N':
      word = pos_tag[0][i][0]
      word_ner = annotator.ner(word.replace('_', ' '))
      if word_ner[0][0][1] != 'B-LOC' and word_ner[0][0][1] != 'B-PER' and word_ner[0][0][1] != 'B-ORG':
        if word in noun_and_question_list.keys():
          noun_and_question_list[word].append(word_segment(question))
        else:
          noun_and_question_list[word] = [word_segment(question)]
    
    if tag == 'A':
      word = pos_tag[0][i][0]
      if word in adj_and_question_list.keys():
        adj_and_question_list[word].append(word_segment(question))
      else:
        adj_and_question_list[word] = [word_segment(question)]

    if tag == 'V':
      word = pos_tag[0][i][0]
      if word in verb_and_question_list.keys():
        verb_and_question_list[word].append(word_segment(question))
      else:
        verb_and_question_list[word] = [word_segment(question)]

    if tag == 'Np':
      word = pos_tag[0][i][0]
      if word in np_and_question_list.keys():
        np_and_question_list[word].append(word_segment(question))
      else:
        np_and_question_list[word] = [word_segment(question)]

    if tag == 'Nc':
      word = pos_tag[0][i][0]
      if word in nc_and_question_list.keys():
        nc_and_question_list[word].append(word_segment(question))
      else:
        nc_and_question_list[word] = [word_segment(question)]

    if tag == 'P':
      word = pos_tag[0][i][0]
      if word in p_and_question_list.keys():
        p_and_question_list[word].append(word_segment(question))
      else:
        p_and_question_list[word] = [word_segment(question)]

    if tag == 'B':
      word = pos_tag[0][i][0]
      if word in b_and_question_list.keys():
        b_and_question_list[word].append(word_segment(question))
      else:
        b_and_question_list[word] = [word_segment(question)]

100%|██████████| 30768/30768 [09:53<00:00, 51.81it/s]


In [ ]:
all_word_list = {
    'noun': [],
    'np': [],
    'nc': [],
    'p': [],
    'b': [],
    'adj': [],
    'verb': [],
}

for key in noun_and_question_list.keys():
  all_word_list['noun'].append(key)

for key in np_and_question_list.keys():
  all_word_list['np'].append(key)

for key in nc_and_question_list.keys():
  all_word_list['nc'].append(key)

for key in b_and_question_list.keys():
  all_word_list['b'].append(key)

for key in p_and_question_list.keys():
  all_word_list['p'].append(key)

for key in adj_and_question_list.keys():
  all_word_list['adj'].append(key)

for key in verb_and_question_list.keys():
  all_word_list['verb'].append(key)

In [ ]:
import json
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/all_word_list.json","w", encoding='utf-8') as jsonfile:
    json.dump(all_word_list,jsonfile,ensure_ascii=False, indent=2)

NameError: ignored

In [ ]:
noun_and_question_list.keys()

dict_keys(['tổng_sản_phẩm', 'năm', 'nội_địa', 'Chỉ_số', 'Tổng_sản_phẩm', 'Cộng_hoà_Trung_Phi', 'chỉ_số', 'khu_vực', 'Tổng', 'giá_trị', 'nước', 'Thành_phố', 'Ma', 'tỉ_lệ', 'triệu_phú', 'Năm', 'đầu', 'người', 'thu_nhập', 'Thu_nhập', 'khoảng', 'châu', 'mã_số', 'điện_thoại', 'Mã_số', 'quốc_tế', 'Nước', 'quốc_gia', 'Mã', 'dân_số', 'số', 'dân', 'Tỉnh', 'Dân_số', 'tỉnh', 'hiện_nay', 'Huyện', 'hiện_tại', 'thời_điểm', 'đất_nước', 'Hiện', 'Số', 'Tổng_số', 'huyện', 'tháng', 'Cộng_hoà', 'tp', 'thành_phố', 'thế_giới', 'cuối', 'dân_chính', 'xác', 'vịnh', 'cộng_hoà', 'nhân_dân', 'Hiện_nay', 'quận', 'vùng', 'đô_thị', 'Hiện_tại', 'núi', 'vương_quốc', 'Quận', 'hiện', 'Vịnh', 'tổng', 'thủ_đô', 'dân_cư', 'Thủ_đô', 'đồng_bằng', 'sông', 'Dân_chủ', 'mo', 'Số_lượng', 'đân', 'đảo', 'Mật_độ', 'mật_độ', 'Thế_giới', 'Múi_giờ', 'múi_giờ', 'thứ', 'múi', 'Tác_giả', 'bộ', 'truyện_tranh', 'Ngư', 'tiều', 'y_thuật', 'tác_phẩm', 'Người', 'tác_giả', 'tiểu_thuyết', 'quốc_ca', 'hoạ', 'hồ', 'thiên_nga', 'Quốc_ca', 'tranh', '

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/noun_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/adj_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/verb_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/np_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(np_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/nc_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(nc_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/b_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(b_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/p_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(p_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
len(noun_and_question_list.keys())

4237

In [ ]:
noun_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/noun_and_question.json')
data = json.load(noun_file)

for key in data.keys():
  if '\x1d' in key:
    key = key.replace('\x1d','')
    for i in range(len(data[key])):
      data[key][i] = data[key][i].replace('\x1d','')

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='vinai/phobert-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
list_of_pos_synonym = {}
for key in tqdm(data.keys()):
  question_word_dict = {}
  for sentence in data[key]:
    raw_sentence = sentence
    mask_sentence = sentence.replace(key, '<mask>', 1)
    unmasked_result = unmasker(mask_sentence, top_k=50)
    
    for i in range(50):
      word = unmasked_result[i]['token_str']
      prob = unmasked_result[i]['score']
      if sentence in question_word_dict.keys():
        if word not in question_word_dict[sentence] and word not in punctuation_set and '@' not in word:
          question_word_dict[sentence].append([word, prob])
      else:
        question_word_dict[sentence] = [[word, prob]]

  list_of_pos_synonym[key] = question_word_dict  

  0%|          | 10/4237 [03:38<8:12:48,  7.00s/it]

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/list_of_pos_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(list_of_pos_synonym,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
import json
pos_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/list_of_pos_synonym.json', encoding='utf-8')
pos_synonym_list = json.loads(pos_synonym_file.read())

In [ ]:
len(pos_synonym_list['dân_chính']['Số dân_chính xác của nước Việt_Nam'])

46

In [ ]:
pos_synonym_list.keys()

dict_keys(['tổng_sản_phẩm', 'năm', 'nội_địa', 'Chỉ_số', 'Tổng_sản_phẩm', 'Cộng_hoà_Trung_Phi', 'chỉ_số', 'khu_vực', 'Tổng', 'giá_trị', 'nước', 'Thành_phố', 'Ma', 'tỉ_lệ', 'triệu_phú', 'Năm', 'đầu', 'người', 'thu_nhập', 'Thu_nhập', 'khoảng', 'châu', 'mã_số', 'điện_thoại', 'Mã_số', 'quốc_tế', 'Nước', 'quốc_gia', 'Mã', 'dân_số', 'số', 'dân', 'Tỉnh', 'Dân_số', 'tỉnh', 'hiện_nay', 'Huyện', 'hiện_tại', 'thời_điểm', 'đất_nước', 'Hiện', 'Số', 'Tổng_số', 'huyện', 'tháng', 'Cộng_hoà', 'tp', 'thành_phố', 'thế_giới', 'cuối', 'dân_chính', 'xác', 'vịnh', 'cộng_hoà', 'nhân_dân', 'Hiện_nay', 'quận', 'vùng', 'đô_thị', 'Hiện_tại', 'núi', 'vương_quốc', 'Quận', 'hiện', 'Vịnh', 'tổng', 'thủ_đô', 'dân_cư', 'Thủ_đô', 'đồng_bằng', 'sông', 'Dân_chủ', 'mo', 'Số_lượng', 'đân', 'đảo', 'Mật_độ', 'mật_độ', 'Thế_giới', 'Múi_giờ', 'múi_giờ', 'thứ', 'múi', 'Tác_giả', 'bộ', 'truyện_tranh', 'Ngư', 'tiều', 'y_thuật', 'tác_phẩm', 'Người', 'tác_giả', 'tiểu_thuyết', 'quốc_ca', 'hoạ', 'hồ', 'thiên_nga', 'Quốc_ca', 'tranh', '

In [ ]:
class_names = ['False', 'True']

def infer(text1, text2, tokenizer, max_len=128):
    #text1 = word_segment(text1)
    #text2 = word_segment(text2)
    encoded_review = tokenizer(
        text1,
        text2,
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    #print(f'Text1: {text1}')
    #print(f'Text2: {text2}')
    #print(f'Is_equivalence: {class_names[y_pred]}')
    return y_pred.item()

In [ ]:
class_names = ['False', 'True']

def infer1(tmp_dict, tokenizer, max_len=128):
    #text1 = word_segment(text1)
    #text2 = word_segment(text2)
    encoded_review = tokenizer(
        tmp_dict['text1'],
        tmp_dict['text2'],
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    #print(f'Text1: {text1}')
    #print(f'Text2: {text2}')
    #print(f'Is_equivalence: {class_names[y_pred]}')
    return y_pred

In [ ]:
synonym_dict = {}

batch_sentence = {
    'text1': [],
    'text2': []
}

word_and_prob = {
    'word': [],
    'prob': [],
    'sentence': []
}

batch_size = 0
cnt = 0

for key in tqdm(pos_synonym_list.keys()):
  small_synonym_dict = {}
  for sub_key in pos_synonym_list[key]:
    for i in range(len(pos_synonym_list[key][sub_key])):
      pos_synonym_word = pos_synonym_list[key][sub_key][i][0]
      if pos_synonym_word in punctuation_set:
        continue
      
      synonym_score = pos_synonym_list[key][sub_key][i][1]
      sentence = sub_key.replace(key, pos_synonym_word)

      if batch_size < 16:
        batch_sentence['text1'].append(sub_key)
        batch_sentence['text2'].append(sentence)
        word_and_prob['word'].append(pos_synonym_word)
        word_and_prob['prob'].append(synonym_score)
        word_and_prob['sentence'].append(sub_key)

        batch_size += 1
        cnt += 1
      else:
        batch_size = 0
        list_similarity_score = infer1(batch_sentence, tokenizer)
        batch_sentence['text1'] = []
        batch_sentence['text2'] = []

        for k in range(len(list_similarity_score.tolist())):
          similarity_score = list_similarity_score.tolist()[k]
          if similarity_score == 1:
            if word_and_prob['sentence'][k] in small_synonym_dict.keys():
              if word_and_prob['word'][k] not in small_synonym_dict[word_and_prob['sentence'][k]].keys():
                small_synonym_dict[word_and_prob['sentence'][k]][word_and_prob['word'][k]] = word_and_prob['prob'][k]
            else:
              small_synonym_dict[word_and_prob['sentence'][k]] = {word_and_prob['word'][k]: word_and_prob['prob'][k]}

        word_and_prob['word'] = []
        word_and_prob['prob'] = []
        word_and_prob['sentence'] = []
    
    if batch_size > 0:
      batch_size = 0
      list_similarity_score = infer1(batch_sentence, tokenizer)
      batch_sentence['text1'] = []
      batch_sentence['text2'] = []

      for k in range(len(list_similarity_score.tolist())):
        similarity_score = list_similarity_score.tolist()[k]
        if similarity_score == 1:
          if word_and_prob['sentence'][k] in small_synonym_dict.keys():
            if word_and_prob['word'][k] not in small_synonym_dict[word_and_prob['sentence'][k]].keys():
              small_synonym_dict[word_and_prob['sentence'][k]][word_and_prob['word'][k]] = word_and_prob['prob'][k]
          else:
            small_synonym_dict[word_and_prob['sentence'][k]] = {word_and_prob['word'][k]: word_and_prob['prob'][k]}

      word_and_prob['word'] = []
      word_and_prob['prob'] = []
      word_and_prob['sentence'] = []
    synonym_dict[key] = small_synonym_dict



100%|██████████| 4237/4237 [5:08:18<00:00,  4.37s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/list_of_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_dict,jsonfile,ensure_ascii=False, indent=2)

In [10]:
import json
synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/list_of_synonym.json', encoding='utf-8')
synonym_dict = json.loads(synonym_file.read())

In [12]:
synonym_dict.keys()

dict_keys(['tổng_sản_phẩm', 'năm', 'nội_địa', 'Chỉ_số', 'Tổng_sản_phẩm', 'Cộng_hoà_Trung_Phi', 'chỉ_số', 'khu_vực', 'Tổng', 'giá_trị', 'nước', 'Thành_phố', 'Ma', 'tỉ_lệ', 'triệu_phú', 'Năm', 'đầu', 'người', 'thu_nhập', 'Thu_nhập', 'khoảng', 'châu', 'mã_số', 'điện_thoại', 'Mã_số', 'quốc_tế', 'Nước', 'quốc_gia', 'Mã', 'dân_số', 'số', 'dân', 'Tỉnh', 'Dân_số', 'tỉnh', 'hiện_nay', 'Huyện', 'hiện_tại', 'thời_điểm', 'đất_nước', 'Hiện', 'Số', 'Tổng_số', 'huyện', 'tháng', 'Cộng_hoà', 'tp', 'thành_phố', 'thế_giới', 'cuối', 'dân_chính', 'xác', 'vịnh', 'cộng_hoà', 'nhân_dân', 'Hiện_nay', 'quận', 'vùng', 'đô_thị', 'Hiện_tại', 'núi', 'vương_quốc', 'Quận', 'hiện', 'Vịnh', 'tổng', 'thủ_đô', 'dân_cư', 'Thủ_đô', 'đồng_bằng', 'sông', 'Dân_chủ', 'mo', 'Số_lượng', 'đân', 'đảo', 'Mật_độ', 'mật_độ', 'Thế_giới', 'Múi_giờ', 'múi_giờ', 'thứ', 'múi', 'Tác_giả', 'bộ', 'truyện_tranh', 'Ngư', 'tiều', 'y_thuật', 'tác_phẩm', 'Người', 'tác_giả', 'tiểu_thuyết', 'quốc_ca', 'hoạ', 'hồ', 'thiên_nga', 'Quốc_ca', 'tranh', '

In [25]:
synonym_word = {}

for key in tqdm(synonym_dict.keys()):
  word_occurence = {}
  for sub_key in synonym_dict[key]:
    for word in synonym_dict[key][sub_key]:
      if word in word_occurence:
        word_occurence[word] += 1
      else:
        word_occurence[word] = 1
  word_occurence = {k: v for k, v in sorted(word_occurence.items(), key=lambda item: item[1], reverse=True)}
  synonym_word[key] = word_occurence

100%|██████████| 4237/4237 [00:00<00:00, 12320.80it/s]


In [27]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/occurence_of_word.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_word,jsonfile,ensure_ascii=False, indent=2)

In [98]:
def get_num_occurence(item):
    """Get the sum of Python and JavaScript skill"""
    # Return default value that is equivalent to no skill
    return item[1]['occurence']

In [109]:
synonym_word_sentence = {}

for key in tqdm(synonym_dict.keys()):
  word_occurence = {}
  for sub_key in synonym_dict[key]:
    for word in synonym_dict[key][sub_key]:
      if word in word_occurence:
        word_occurence[word]['occurence'] += 1
        if len(word_occurence[word]['sentence']) < 3:
          sentence = sub_key.replace(key, '<mask>', 1)
          word_occurence[word]['sentence'].append(sentence)
      else:
        sentence = sub_key.replace(key, '<mask>', 1)
        word_occurence[word] = {'occurence': 1, 'sentence': [sentence]}
  word_occurence = {k: v for k, v in sorted(word_occurence.items(), key=get_num_occurence, reverse=True)}
  if len(word_occurence) > 0:
    synonym_word_sentence[key] = word_occurence

100%|██████████| 4237/4237 [00:01<00:00, 3183.09it/s]


In [110]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/occurence_of_word_sentence.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_word_sentence,jsonfile,ensure_ascii=False, indent=2)

In [8]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = '/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/Noun'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file, on_bad_lines='skip', header=None) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)

In [9]:
big_df_list = big_df.values.tolist()

In [11]:
import json
synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/list_of_synonym.json', encoding='utf-8')
synonym_dict = json.loads(synonym_file.read())

In [12]:
cnt=0
word = []
isIncreased=False
for key in tqdm(synonym_dict.keys()):
  detokenized_key = key.replace('_', ' ')
  for list_word in big_df_list:
    if detokenized_key in list_word:
      if isIncreased == False:
        word.append(key)
        cnt+=1
      isIncreased=True
  isIncreased=False

print(cnt)

100%|██████████| 4237/4237 [00:16<00:00, 250.82it/s]

1205


In [13]:
word

['năm',
 'chỉ_số',
 'khu_vực',
 'giá_trị',
 'nước',
 'tỉ_lệ',
 'triệu_phú',
 'đầu',
 'người',
 'thu_nhập',
 'khoảng',
 'mã_số',
 'điện_thoại',
 'Mã',
 'dân_số',
 'số',
 'dân',
 'tỉnh',
 'hiện_tại',
 'thời_điểm',
 'huyện',
 'tháng',
 'thành_phố',
 'thế_giới',
 'cuối',
 'vịnh',
 'quận',
 'vùng',
 'đô_thị',
 'núi',
 'vương_quốc',
 'tổng',
 'thủ_đô',
 'đồng_bằng',
 'sông',
 'đảo',
 'thứ',
 'múi',
 'bộ',
 'truyện_tranh',
 'tác_phẩm',
 'tiểu_thuyết',
 'quốc_ca',
 'hồ',
 'thiên_nga',
 'nhạc_sĩ',
 'tượng',
 'thép',
 'Bộ',
 'truyện',
 'trai',
 'chí',
 'lịch',
 'triều',
 'loại',
 'hiệp',
 'bài',
 'ông_tổ',
 'lời',
 'nhà_thờ',
 'huy_hiệu',
 'tập',
 'thơ',
 'tù',
 'ngôn_ngữ',
 'thư',
 'kịch_bản_phim',
 'sách',
 'ngôn_ngữ_lập_trình',
 'thám_tử',
 'Truyện_Kiều',
 'truyện_ngắn',
 'lá',
 'đèn',
 'đá',
 'phù_thuỷ',
 'ngày',
 'thời_gian',
 'ngày_tháng',
 'chiều',
 'gió',
 'lần',
 'sữa',
 'phim',
 'cốc',
 'lửa',
 'loài',
 'mông',
 'tâm',
 'ý_nghĩa',
 'nội_dung',
 'thuyết_tương_đối',
 'điều',
 'Dư_địa_chí

In [14]:
synonym_dict['bảng_tuần_hoàn']

{'Sắt trong bảng_tuần_hoàn nguyên_tố có ký_hiệu là gì': {'các': 0.3392196297645569,
  'một': 0.05523262172937393,
  'một_số': 0.042399562895298004,
  'mỗi': 0.03321625664830208,
  'bảng': 0.02151305042207241,
  '12': 0.017731256783008575,
  'danh_sách': 0.0145685700699687,
  'từng': 0.013349059037864208,
  'đơn_vị': 0.009762236848473549,
  'thang': 0.0095831835642457,
  'kim_loại': 0.009042621590197086,
  'Danh_mục': 0.00890886690467596,
  'danh_mục': 0.008538763038814068,
  'tên': 0.008100900799036026,
  'Danh_sách': 0.006190301384776831,
  'nhóm': 0.00531575595960021,
  'tiền': 0.00519832968711853,
  '5': 0.004181702621281147,
  'số': 0.003996408078819513,
  'bảng_chữ_cái': 0.0038079554215073586,
  'lue': 0.0037019187584519386,
  'nguyên_tố': 0.0029346486553549767,
  'phép': 0.0029044905677437782,
  'Khoa_học_Tự_nhiên': 0.002897340338677168,
  'nhiều': 0.0028677203226834536,
  'những': 0.0028004616033285856,
  'hệ': 0.002705588238313794,
  'Các': 0.0026757712475955486,
  'tổng_số': 0